# Store Aisle Monitor

This reference implementation counts the number of people present in an image. It takes the input from a video file for processing. Snapshots of the output are taken at regular intervals and are uploaded to the cloud. It also stores the snapshots of the output locally.

## Overview of how it works
At start-up the sample application reads the equivalent of command line arguments and loads a network and image from the video input to the Inference Engine (IE) plugin. A job is submitted to an edge compute node with a hardware accelerator such as Intel® HD Graphics GPU, Intel® Movidius™ Neural Compute Stick 2 and Intel® Arria® 10 FPGA.
After the inference is completed, the output videos are appropriately stored in the /results/[device] directory, which can then be viewed within the Jupyter Notebook instance.

## Demonstration objectives
* Video as input is supported using **OpenCV**
* Inference performed on edge hardware (rather than on the development node hosting this Jupyter notebook)
* **OpenCV** provides the bounding boxes, labels and other information
* Visualization of the resulting bounding boxes


## Step 0: Set Up

### 0.1: Import dependencies

Run the below cell to import Python dependencies needed for displaying the results in this notebook
(tip: select the cell and use **Ctrl+enter** to run the cell)

In [1]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

### 0.2  (Optional-step): Original video without inference

If you are curious to see the input video, run the following cell to view the original video stream used for inference and store aisle monitor.

In [2]:
!ln -sf ./resources/store-aisle-detection.mp4
videoHTML('Store Aisle Video', ['store-aisle-detection.mp4 '])

### 0.3 Install Azure
To run application on azure cloud `azure` should be installed in the system. Run following commands to install azure

In [6]:
try:
    import azure
    print("Azure already installed")
except:
    print("Installing Azure")
    !pip3 install azure

Installing Azure
  Using cached https://files.pythonhosted.org/packages/5a/5e/f94bd8091ab0ccc68ee902e188d4e8cd3b38dc77155a1c187cbf4767d0e4/azure-4.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/72/94/4db044f1c155b40c5ebc037bfd9d1c24562845692c06798fbe869fe160e6/azure_storage_queue-1.4.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2c/d9/3e33ad19eb57988dff3aac06bab241736f86dc692a38c5ff6bb6fb959243/azure_servicefabric-6.3.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b1/fa/1053b5dcd88e5de8e8cd70a4d7189ffad037542963ea86b518deb612c498/azure_batch-4.1.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/01/e4/2a3319f04c69f0113a05884664769d462dcec55031c6d154a2c4176cfb79/azure_eventgrid-1.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/55/81/f57fea1cdab4490bc185e48a4db2451795fde37eb8fed1808e7e8fd2c99e/azure_cosmosdb_table-1.0.5-py2.py3-none-

  Using cached https://files.pythonhosted.org/packages/65/2e/6d9649bd5645ac32a179adfea5707f640fc51a57e493c27f0ab575789e85/azure_mgmt_media-1.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/90/a7/42342a984b2916972c5c5e24df94e3cd5e4377a8dc465a83415706d9be6f/azure_mgmt_redis-5.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/cb/f3/a86ba3e0784d12c8fe5cbf1f24e1b9255575a2f0892e08c46cddd0795dfd/azure_mgmt_advisor-1.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ae/95/b451721e38ca0feddce03ee3ce86158e208d0150253bd371207a8df4e9c5/azure_mgmt_msi-0.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/ad/9e90f8bab40a9682410e57ed08a799be113c5e470bec247b099038c6389e/azure_mgmt_servicebus-0.5.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/73/e7/5d909ef5812fc31f2871f3510ef43af93157ba51be03b6f798afba7a29d8/azure_mgmt_datalake_analytics-0.6.0-py2.py3

  Using cached https://files.pythonhosted.org/packages/97/81/a9eb3fd2ab070159105b4cfe9640c24410ac8195286729d62bfdf871de94/azure_mgmt_batch-5.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fd/7f/80327a27e03473101afb136e872279ad1386adf1bac14a58a9f839cd8352/azure_mgmt_recoveryservices-0.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/19/84/7021951ae69572a19af0ff9cca1d2abf3542d35af38796656ded1be08358/azure_mgmt_policyinsights-0.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fd/d1/d770050f20ad81b80f7eb41f89e1a5d841cf74bf41c7e1ff137c46f28a1e/azure_mgmt_containerinstance-1.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5a/cb/3822bcfa815f894c86a27845ae4160c0169722598399234cd6f77bb82347/azure_mgmt_eventhub-2.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6e/06/fafe8b5d881cfa68927e61557c8419dcfacb93e07f4ab17cc60959707a53/azure_mgmt_se

## Step 1: Using Intel® Distribution of OpenVINO™ toolkit

We will be using Intel® Distribution of OpenVINO™ toolkit Inference Engine (IE) to locate person in the frame.
There are five steps involved in this task:

1. Create an Intermediate Representation (IR) Model using the Model Optimizer by Intel
2. Choose a device and create IEPlugin for the device
3. Read the IRModel using IENetwork
4. Load the IENetwork into the Plugin
5. Run inference.

### 1.1 Creating IR Model

The Model Optimizer creates Intermediate Representation (IR) models that are optimized for different end-point target devices.
These models can be created from existing DNN models from popular frameworks (e.g. Caffe*, TF) using the Model Optimizer. 
The Intel® Distribution of OpenVINO™ toolkit includes a utility script `model_downloader.py` that you can use to download some common models. Run the following cell to see the models available through `model_downloader.py`

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --print_all

**Note**: The '!' is a special Jupyter Notebook command that allows you to run shell commands as if you are in a command line. So the above command will work straight out of the box on in a terminal (with '!' removed).

Some of these downloaded models are already in the IR format, while others will require the model optimizer. In this demo, we will be using the **person-detection-retail-0013** model, which is already in IR format. This model can be downloaded with the following command.

In [4]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name person-detection-retail-0013 -o models
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name person-detection-retail-0013-fp16 -o models


###############|| Downloading topologies ||###############

========= Downloading models/Retail/object_detection/pedestrian/rmnet_ssd/0013/dldt/FP32/person-detection-retail-0013.xml
... 100%, 152 KB, 142 KB/s, 1 seconds passed

========= Downloading models/Retail/object_detection/pedestrian/rmnet_ssd/0013/dldt/FP32/person-detection-retail-0013.bin
... 100%, 2823 KB, 990 KB/s, 2 seconds passed

========= Downloading models/Retail/object_detection/pedestrian/rmnet_ssd/0013/dldt/FP16/person-detection-retail-0013.xml
... 100%, 152 KB, 377 KB/s, 0 seconds passed

========= Downloading models/Retail/object_detection/pedestrian/rmnet_ssd/0013/dldt/FP16/person-detection-retail-0013.bin
... 100%, 1411 KB, 767 KB/s, 1 seconds passed

========= Downloading models/Retail/object_detection/pedestrian/rmnet_ssd/0013/dldt/INT8/person-detection-retail-0013.xml
... 100%, 1950 KB, 693 KB/s, 2 seconds passed

========= Downloading models/Retail/object_detection/pedestrian/rmnet_ssd/0013/dldt/INT8/person-

The input arguments are as follows:
* --name : name of the model you want to download. It should be one of the models listed in the previous cell
* -o : output directory. If this directory does not exist, it will be created for you.

There are more arguments to this script and you can get the full list using the `-h` option.


With the `-o` option set as above, this command downloads the model in the directory `models`, with the model files (.xml and .bin) located at `/Retail/object_detection/pedestrian/rmnet_ssd/0013/dldt`



## Step 2 : Inference on a video

The inference code is already implemented in 
<a href="main.py">main.py</a>.

The Python code takes in command line arguments for video, model etc.

**Command line argument options and how they are interpreted in the application source code**

```
python3 store_aisle_monitor.py -m ${MODELPATH} \
                -i ${INPUT_FILE} \
                -o ${OUTPUT_FILE} \
                -d ${DEVICE} \
                -pt ${THRESHOLD} \
                -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_sse4.so
```
To upload the results to the cloud, the Microsoft Azure storage name and storage key are provided as the command line arguments. Use `-an` and `-ak` options to specify Microsoft Azure storage name and storage key respectively.

```
python3 store_aisle_monitor.py -m ${MODELPATH} \
                -i ${INPUT_FILE} \
                -o ${OUTPUT_FILE} \
                -d ${DEVICE} \
                -pt ${THRESHOLD} \
                -an <azure-account-name> \
                -ak <azure-account-key> \
                -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_sse4.so

```
##### The description of the arguments used in the argument parser is the command line executable equivalent.
* -m location of the pre-trained IR model which has been pre-processed using the model optimizer. There is automated support built in this argument to support both FP32 and FP16 models targeting different hardware

* -i  location of the input video stream
* -o location where the output file with inference needs to be stored (results/[device])
* -d type of Hardware Acceleration (CPU, GPU, MYRIAD, HDDL or HETERO:FPGA,CPU)
* -pt probability threshold value for the person detection
* -l absolute path to the shared library and is currently optimized for core/xeon (/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_sse4.so)
* -an microsoft azure storage name
* -ak microsoft azure storage key

### 2.1 Creating job file
 
To run inference on the video, we need more compute power.
We will run the workload on several edge compute nodes represented in the IoT DevCloud. We will send work to the edge compute nodes by submitting the corresponding non-interactive jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

The job file is written in Bash, and will be executed directly on the edge compute node.
For this example, we have written the job file for you in the notebook.
Run the following cell to write this in to the file "store_aisle_job.sh"

In [7]:
#Running the store aisle code
!python3 store_aisle_monitor.py  -m models/Retail/object_detection/pedestrian/rmnet_ssd/0013/dldt/FP32/person-detection-retail-0013.xml -i resources/store-aisle-detection.mp4 -o results -d CPU -pt 0.7 -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_sse4.so

Traceback (most recent call last):
  File "store_aisle_monitor.py", line 30, in <module>
    from azure.storage.blob import BlockBlobService, PublicAccess
ImportError: No module named 'azure'


### 2.2 Understand how jobs are submitted into the queue

Now that we have the job script, we can submit the jobs to edge compute nodes. In the IoT DevCloud, you can do this using the `qsub` command.
We can submit store_aisle_job to several different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command that we use for this:
- `-l` : this option let us select the number and the type of nodes using `nodes={node_count}:{property}`. 
- `-F` : this option let us send arguments to the bash script. 
- `-N` : this option let us name the job so that it is easier to distinguish between them.

The `-F` flag is used to pass in arguments to the job script.
The [store_aisle_job.sh](store_aisle_job.sh) takes in 5 arguments:
1. the path to the directory for the output video and performance stats
2. targeted device (e.g. CPU, GPU, MYRIAD, HDDL or HETERO:FPGA,CPU)
3. the floating precision to use for inference
4. location of the input video stream
5. probability threshold value for the person detection

The job scheduler will use the contents of `-F` flag as the argument to the job script.

If you are curious to see the available types of nodes on the IoT DevCloud, run the following optional cell.

In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

Here, the properties describe the node, and number on the left is the number of available nodes of that architecture.

**Note**: If you want to use your own video, change the environment variable 'VIDEO' in the following cell from "resources/store-aisle-detection.mp4" to the full path of your uploaded video.

In [ ]:
os.environ["VIDEO"] = "resources/store-aisle-detection.mp4"

### 2.3 Job queue submission

Each of the cells below will submit a job to different edge compute nodes.
The output of the cell is the `JobID` of your job, which you can use to track progress of a job.

**Note** You can submit all jobs at once or follow one at a time. 

After submission, they will go into a queue and run as soon as the requested compute resources become available. 
(tip: **shift+enter** will run the cell and automatically move you to the next cell. So you can hit **shift+enter** multiple times to quickly run multiple cells)

#### Submitting to an edge compute node with an Intel® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core™ i5-6500TE processor</a>. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_core = !qsub store_aisle_job.sh -l nodes=1:tank-870:i5-6500te -F "results/core/ CPU FP32 $VIDEO 0.7" -N store_core
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/core/', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)

#### Submitting to an edge compute node with Intel® Xeon® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel® 
    Xeon® Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon = !qsub store_aisle_job.sh  -l nodes=1:tank-870:e3-1268l-v5 -F "results/xeon/ CPU FP32 $VIDEO 0.7" -N store_xeon 
print(job_id_xeon[0]) 
#Progress indicators
if job_id_xeon:
    progressIndicator('results/xeon/', 'i_progress_'+job_id_xeon[0]+'.txt', "Inference", 0, 100)

#### Submitting to an edge compute node with Intel® Core CPU and using the onboard Intel® GPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core i5-6500TE</a>. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub store_aisle_job.sh -l nodes=1:tank-870:i5-6500te:intel-hd-530 -F "results/gpu/ GPU FP32 $VIDEO 0.7" -N store_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('results/gpu/', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)

#### Submitting to an edge compute node with Intel® NCS 2 (Neural Compute Stick 2)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://software.intel.com/en-us/neural-compute-stick">Intel Neural Compute Stick 2</a> installed in this  node.

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub store_aisle_job.sh -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "results/ncs2/ MYRIAD FP16 $VIDEO 0.7" -N store_ncs2
print(job_id_ncs2[0]) 
#Progress indicators
if job_id_ncs2:
    progressIndicator('results/ncs2/', 'i_progress_'+job_id_ncs2[0]+'.txt', "Inference", 0, 100)

#### Submitting to an edge compute node with UP Squared Grove IoT Development Kit (UP2)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/up-squared-grove-dev-kit">UP Squared Grove IoT Development Kit</a> edge node with an <a 
    href="https://ark.intel.com/products/96488/Intel-Atom-x7-E3950-Processor-2M-Cache-up-to-2-00-GHz-">Intel® Atom® x7-E3950 Processor</a>. The inference  workload will run on the integrated Intel® HD Graphics 505 card.

In [ ]:
#Submit job to the queue
job_id_up2 = !qsub store_aisle_job.sh -l nodes=1:up-squared -F "results/up2/ GPU FP32 $VIDEO 0.7" -N store_up2
print(job_id_up2[0]) 
#Progress indicators
if job_id_up2:
    progressIndicator('results/up2/', 'i_progress_'+job_id_up2[0]+'.txt', "Inference", 0, 100)

#### Submitting to an edge compute node with IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core™ i5-6500te CPU</a> . The inference workload will run on the <a href="https://www.ieiworld.com/mustang-f100/en/"> IEI Mustang-F100-A10 </a> card installed in this node.

In [ ]:
#Submit job to the queue
job_id_fpga = !qsub store_aisle_job.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-f100-a10 -F "results/fpga/ HETERO:FPGA,CPU FP32 $VIDEO 0.7" -N store_fpga
print(job_id_fpga[0]) 
#Progress indicators
if job_id_fpga:
    progressIndicator('results/fpga/', 'i_progress_'+job_id_fpga[0]+'.txt', "Inference", 0, 100)

### 2.4 Check if the jobs are done

To check on the jobs that were submitted, use the `qstat` command.

We have created a custom Jupyter widget  to get live qstat update.
Run the following cell to bring it up.

In [ ]:
liveQstat()

You should see the jobs you have submitted (referenced by `Job ID` that gets displayed right after you submit the job in step 2.3).
There should also be an extra job in the queue "jupyterhub": this job runs your current Jupyter Notebook session.

The 'S' column shows the current status. 
- If it is in Q state, it is in the queue waiting for available resources. 
- If it is in R state, it is running. 
- If the job is no longer listed, it means it is completed.

**Note**: Time spent in the queue depends on the number of users accessing the edge nodes. Once these jobs begin to run, they should take from 1 to 5 minutes to complete.

***Wait!***

Please wait for the inference jobs and video rendering to complete before proceeding to the next step.

## Step 3: View Results

Once the jobs are completed, the queue system outputs the stdout and stderr streams of each job into files with names of the form

`store_{type}.o{JobID}`

`store_{type}.e{JobID}`

(here, store_{type} corresponds to the `-N` option of qsub).

However, for this case, we may be more interested in the output video files. They are stored in mp4 format inside the `results/[device]` directory.
We wrote a short utility script that will display these videos within the notebook.
Run the cells below to display them.
See `demoutils.py` if you are interested in understanding further on how the results are displayed in notebook.

In [ ]:
videoHTML('IEI Tank (Intel Core CPU)', 
         ['results/core/store_aisle.mp4'],'results/core/stats.txt')

In [ ]:
videoHTML('IEI Intel GPU (Intel Core + Onboard GPU)', 
          ['results/gpu/store_aisle.mp4'],'results/gpu/stats.txt')

In [ ]:
videoHTML('IEI Tank + Intel CPU + Intel NCS2', 
          ['results/ncs2/store_aisle.mp4'],'results/ncs2/stats.txt')

In [ ]:
videoHTML('IEI Tank Xeon (Intel Xeon CPU)', 
          ['results/xeon/store_aisle.mp4'],'results/xeon/stats.txt')

In [ ]:
videoHTML('UP Squared Grove IoT Development Kit (UP2)', 
          ['results/up2/store_aisle.mp4'],'results/up2/stats.txt')

In [ ]:
videoHTML('IEI Tank + IEI Mustang-F100-A10 (Intel® Arria® 10 FPGA)', 
          ['results/fpga/store_aisle.mp4'],'results/fpga/stats.txt')

## Step 4: Assess Performance

The running time of each inference task is recorded in `results/[device]/stats.txt`. Run the cell below to plot the results of all jobs side-by-side. Lower values mean better performance for **Inference Engine Processing Time**. Keep in mind that some architectures are optimized for the highest performance, others for low power or other metrics.

In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
             ('fpga', 'IEI Mustang\nF100-A10\nFPGA'),
             ('ncs2', 'Intel\nNCS2'),
             ('up2', 'Intel Atom\nx7-E3950\nUP2/GPU')]

stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        stats_list.append(('results/'+arch+'/stats'+'.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))

summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time' )

summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps' )